In [31]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
import pandas as pd
from numpy import mean, std
print("It's alive")

It's alive


In [55]:
def get_data(url, drop=[]):
  import pandas as pd
  df = pd.read_json(url)
  if len(drop) > 0:
    for col in drop:
      df.drop(columns=[col], inplace=True)
  return df

def bin_groups(df, percent=.05):
  import pandas as pd
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      for group, count in df[col].value_counts().iteritems():
        if count / len(df) < percent:
          df.loc[df[col] == group, col] = 'Other'
  return df

def drop_columns_missing_data(df, cutoff=.5):
  import pandas as pd
  for col in df:
    if df[col].isna().sum() / len(df) > cutoff:
      df.drop(columns=[col], inplace=True)
  return df

def impute_mean(df):
  from sklearn.impute import SimpleImputer
  import pandas as pd, numpy as np
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      df = pd.get_dummies(df, columns=[col], drop_first=True)
  imp = SimpleImputer(missing_values=np.nan, strategy='mean')
  df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
  return df

def impute_KNN(df):
  from sklearn.impute import KNNImputer
  from sklearn.preprocessing import MinMaxScaler
  import pandas as pd
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      df = pd.get_dummies(df, columns=[col], drop_first=True)
  df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns = df.columns)
  imp = KNNImputer(n_neighbors=5, weights="uniform")
  df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
  return df
      
def impute_reg(df):
  from sklearn.experimental import enable_iterative_imputer
  from sklearn.impute import IterativeImputer
  import pandas as pd
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      df = pd.get_dummies(df, columns=[col], drop_first=True)
  imp = IterativeImputer(max_iter=10, random_state=12345)
  df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
  return df

def fs_variance(df, label="", p=0.8):
  from sklearn.feature_selection import VarianceThreshold
  import pandas as pd

  if label != "":
    X = df.drop(columns=[label])
    
  sel = VarianceThreshold(threshold=(p * (1 - p)))
  sel.fit_transform(X)

  # Add the label back in after removing poor features
  return df[sel.get_feature_names_out()].join(df[label])

def fit_crossvalidate_mlr(df, k, label, repeat=True):
  from sklearn.linear_model import LinearRegression
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  import pandas as pd
  from numpy import mean, std
  X = df.drop(label,axis=1)
  y = df[label]
  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)
  print(f'Average R-squared:\t{mean(scores)}')
  return LinearRegression().fit(X, y)

def dump_pickle(model, file_name):
  import pickle
  pickle.dump(model, open(file_name, "wb"))

def load_pickle(file_name):
  import pickle
  model = pickle.load(open(file_name, "rb"))
  return model

In [33]:
df = get_data('https://opendata.utah.gov/resource/herb-zqda.json', [])
df.head()

,crash_id,crash_datetime,route,milepoint,lat_utm_y,long_utm_x,main_road_name,city,county_name,crash_severity_id,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,11281387,2019-02-08T10:56:00.000,2258.0,4.560,4513124.085,422620.0512,900 WEST,SALT LAKE CITY,SALT LAKE,2,...,False,False,True,False,False,False,False,False,False,True
1,11282089,2019-12-24T12:48:00.000,2082.0,2.702,4483700.743,427458.6694,1215 E HIGHLAND DR,DRAPER,SALT LAKE,1,...,False,False,False,True,False,False,False,False,False,False
2,11282092,2019-12-28T20:42:00.000,2062.0,0.351,4486609.366,426779.0152,1022 E DRAPER PKY,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False
3,11282098,2019-12-07T00:46:00.000,2048.0,0.438,4484350.913,426397.6578,13463 S FORT ST,DRAPER,SALT LAKE,3,...,False,True,False,False,False,True,True,False,False,False
4,11282101,2019-12-18T17:29:00.000,2068.0,0.796,4485217.604,427714.0392,13043 S 1300 E,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False


In [34]:
# Call the function to update the DataFrame
df = bin_groups(df, 0.02)

# Check the value_counts() to see if it worked
for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(df[col].value_counts())
        print()

Other    1000
Name: crash_datetime, dtype: int64

Other    891
I-15     109
Name: main_road_name, dtype: int64

Other                  383
OUTSIDE CITY LIMITS    139
DRAPER                  79
SANDY                   66
SALT LAKE CITY          65
WEST VALLEY CITY        61
WEST JORDAN             36
PROVO                   35
MURRAY                  33
ST. GEORGE              30
LAYTON                  29
LEHI                    23
SOUTH JORDAN            21
Name: city, dtype: int64

SALT LAKE     474
UTAH          140
Other         120
DAVIS          95
WEBER          59
WASHINGTON     44
BOX ELDER      36
CACHE          32
Name: county_name, dtype: int64



In [35]:
print("Feature\t", "Percent Missing\n")

for col in df:
    if df[col].isnull().sum() / len(df) > .5:
        print(f'{col}\t{round(df[col].isnull().sum() / len(df), 4) * 100}%')

Feature	 Percent Missing



In [37]:
# model = fit_crossvalidate_mlr(df, 5, 'crash_severity_id', True)
df.dtypes

#Dummy Codes for all remaining categorical values
for col in df.columns:
    if not pd.api.types.is_numeric_dtype(df[col]):
         df = pd.get_dummies(df, columns=[col], prefix=col)

for col in df.columns:
    print(col)

# print(df)

crash_id
route
milepoint
lat_utm_y
long_utm_x
crash_severity_id
work_zone_related
pedestrian_involved
bicyclist_involved
motorcycle_involved
improper_restraint
unrestrained
dui
intersection_related
wild_animal_related
domestic_animal_related
overturn_rollover
commercial_motor_veh_involved
teenage_driver_involved
older_driver_involved
night_dark_condition
single_vehicle
distracted_driving
drowsy_driving
roadway_departure
crash_datetime_Other
main_road_name_I-15
main_road_name_Other
city_DRAPER
city_LAYTON
city_LEHI
city_MURRAY
city_OUTSIDE CITY LIMITS
city_Other
city_PROVO
city_SALT LAKE CITY
city_SANDY
city_SOUTH JORDAN
city_ST. GEORGE
city_WEST JORDAN
city_WEST VALLEY CITY
county_name_BOX ELDER
county_name_CACHE
county_name_DAVIS
county_name_Other
county_name_SALT LAKE
county_name_UTAH
county_name_WASHINGTON
county_name_WEBER


In [51]:
df.dropna(inplace=True)
model = fit_crossvalidate_mlr(df, 7, 'crash_severity_id', True)

R-squared scores: 
[ 0.17959354 -0.03383149  0.12853768  0.2226833   0.29213778  0.1108237
 -0.00116305  0.10259803  0.06475041  0.06229597  0.16958027  0.20671238
  0.1761322   0.1216755  -0.00228994  0.05822365  0.10617973  0.05570843
  0.19477251  0.075939    0.36007282  0.16280621  0.20744408  0.06263825
  0.04887081  0.09282603  0.07454248  0.18635593  0.02006952  0.04727372
  0.10430834  0.17391622  0.11750247  0.15356797  0.13937312]

Average R-squared:	0.12121793100139318


In [ ]:
model = fit_crossvalidate_mlr(df, 7, 'crash_severity_id', True)

In [56]:
# %pip install xgboost
def fit_crossvalidate_clf(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
  from sklearn import svm
  from sklearn.naive_bayes import CategoricalNB
  from xgboost import XGBClassifier
  from sklearn import preprocessing
  from sklearn.neural_network import MLPClassifier

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  
  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models
  
  # Create the model objects
  model_log = lm.LogisticRegression(max_iter=100)
  model_logcv = lm.RidgeClassifier()
  model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3)
  model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=12345, tol=1e-3)
  model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
  model_knn = KNeighborsClassifier(n_neighbors=3)
  model_svm = svm.SVC(decision_function_shape='ovo') # Remove the parameter for two-class model
  model_nb = CategoricalNB()
  model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
  model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=12345)
  model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=12345)
  model_rf = se.RandomForestClassifier(n_estimators=10)
  model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
  model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
  model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=12345)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
  final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=12345)
  model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
  model_xgb = XGBClassifier()
  model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=12345)

  # Fit a cross-validated R squared score and add it to the dict
  fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))

  # Add the model to another dictionary; make sure the keys have the same names as the list above
  models['Logistic'] = model_log
  models['Ridge'] = model_logcv
  models['SGD'] = model_sgd
  models['PassiveAggressive'] = model_pa
  models['Perceptron'] = model_per
  models['KNN'] = model_knn
  models['SVM'] = model_svm
  models['NaiveBayes'] = model_nb
  models['Bagging'] = model_bag
  models['AdaBoost'] = model_ada
  models['ExtraTrees'] = model_ext
  models['RandomForest'] = model_rf
  models['HistGradient'] = model_hgb
  models['Voting'] = model_vot
  models['GradBoost'] = model_gb
  models['XGBoost'] = model_xgb
  models['NeuralN'] = model_nn

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'Accuracy':fit})
  df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

# Data cleaning and preparation pipeline
df_new = get_data('https://opendata.utah.gov/resource/herb-zqda.json', [])
df_new.head()
df_new = bin_groups(df_new)
df_new = drop_columns_missing_data(df_new)

# Drop the label so it does not get dummy-coded, then join it back in after
df_new = impute_mean(df_new.drop(columns=["crash_severity_id"])).join(df_new.crash_severity_id)

# Feature selection and modeling pipeline
df_new = fs_variance(df_new, label="crash_severity_id", p=.5)
model = fit_crossvalidate_clf(df_new, "crash_severity_id", 5, 2)

# Deployment pipeline
dump_pickle(model, 'best_clf_model.sav')

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scipy in c:\users\whyri\onedrive\desktop\one_folder\stuff\new455\ml-venv2\lib\site-packages (from xgboost) (1.8.0)



c:\Users\whyri\OneDrive\Desktop\One_Folder\stuff\New455\ml-venv2\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
c:\Users\whyri\OneDrive\Desktop\One_Folder\stuff\New455\ml-venv2\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\whyri\OneDrive\Desktop\One_Folder\stuff\New455\ml-venv2\lib\site-packages\sklearn\model_selection\_validation.py", line 680

                   Accuracy
Ridge                0.7140
SVM                  0.7140
Logistic             0.7130
Bagging              0.7115
Voting               0.6790
KNN                  0.6690
PassiveAggressive    0.6645
RandomForest         0.6505
HistGradient         0.6500
XGBoost              0.6455
ExtraTrees           0.6355
GradBoost            0.6235
SGD                  0.5340
AdaBoost             0.4245
NeuralN              0.1740
Perceptron           0.1150
NaiveBayes              NaN


In [65]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

df_new.drop_duplicates(inplace=True)
coefs = pd.DataFrame(
    model.coef_,
    columns=['Coefficients'], index=df_new.drop(columns=['crash_severity_id']).columns
)

coefs.sort_values(by=['Coefficients'], ascending=False, inplace=True)

coefs.plot(kind='barh', figsize=(10, 30))
plt.title('MLR model')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

ValueError: Shape of passed values is (5, 5), indices imply (5, 1)